# Modelos de Clasificación – Nivel de gasto del cliente

## 1) Objetivo
Entrenar y evaluar modelos de Machine Learning que permitan clasificar
a los clientes según su nivel de gasto (Bajo, Medio o Alto), apoyando
decisiones de segmentación y estrategias comerciales.

## 2) Importación de librerías

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

## 3) Carga del dataset procesado

In [7]:
df = pd.read_csv('../data/processed/clientes_clasificacion.csv')
df.head()

,Edad,Genero,Categoria_Producto,Metodo_Pago,Shopping_Mall,Nivel_Gasto
0,28,Female,Clothing,Credit Card,Kanyon,Alto
1,21,Male,Shoes,Debit Card,Forum Istanbul,Alto
2,20,Male,Clothing,Cash,Metrocity,Medio
3,66,Female,Shoes,Credit Card,Metropol AVM,Alto
4,53,Female,Books,Cash,Kanyon,Bajo


## 4) Definición de variables predictoras y variable objetivo

In [8]:
X = df.drop('Nivel_Gasto', axis=1)
y = df['Nivel_Gasto']

## 5) Identificación de variables numéricas y categóricas

In [9]:
num_features = ['Edad']
cat_features = [
    'Genero',
    'Categoria_Producto',
    'Metodo_Pago',
    'Shopping_Mall'
]

## 6) Preprocesamiento de datos

- Escalado de variables numéricas
- One-Hot Encoding de variables categóricas

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

## 7) División del dataset

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 8.1) Modelo 1 – Logistic Regression

In [12]:
log_reg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000
    ))
])

log_reg_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

## 8.1.1) Evaluación del modelo – Logistic Regression

In [13]:
y_pred_log = log_reg_pipeline.predict(X_test)

accuracy_log = accuracy_score(y_test, y_pred_log)
print(f"Accuracy: {accuracy_log:.3f}")

Accuracy: 0.723


In [14]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

        Alto       0.66      1.00      0.80      6560
        Bajo       0.96      0.71      0.81      6729
       Medio       0.61      0.46      0.53      6603

    accuracy                           0.72     19892
   macro avg       0.74      0.72      0.71     19892
weighted avg       0.74      0.72      0.71     19892



In [15]:
confusion_matrix(y_test, y_pred_log)

array([[6560,    0,    0],
       [   0, 4751, 1978],
       [3333,  206, 3064]])

La regresión logística permite explicar de forma clara cómo las variables
influyen en la probabilidad de que un cliente pertenezca a cada nivel de gasto.

## 8.2) Modelo 2 – Random Forest Classifier

In [16]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
    ))
])

rf_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

## 8.2.1) Evaluación del modelo – Random Forest

In [17]:
y_pred_rf = rf_pipeline.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf:.3f}")


Accuracy: 0.675


In [18]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

        Alto       0.67      0.81      0.74      6560
        Bajo       0.82      0.78      0.80      6729
       Medio       0.51      0.43      0.47      6603

    accuracy                           0.68     19892
   macro avg       0.67      0.67      0.67     19892
weighted avg       0.67      0.68      0.67     19892



In [19]:
confusion_matrix(y_test, y_pred_rf)

array([[5339,    0, 1221],
       [   0, 5252, 1477],
       [2614, 1147, 2842]])

## 9) Comparación de modelos

In [20]:
pd.DataFrame({
    'Modelo': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [accuracy_log, accuracy_rf]
})

,Modelo,Accuracy
0,Logistic Regression,0.722652
1,Random Forest,0.675297


- Random Forest suele capturar mejor relaciones no lineales.
- Logistic Regression ofrece mayor interpretabilidad.
- El modelo final depende del equilibrio entre precisión y explicabilidad.

## 10) Predicción de gasto – Ejemplos

In [21]:
ejemplos = pd.DataFrame({
    'Edad': [22, 40, 65],
    'Genero': ['Female', 'Male', 'Female'],
    'Categoria_Producto': ['Clothing', 'Technology', 'Shoes'],
    'Metodo_Pago': ['Cash', 'Credit Card', 'Debit Card'],
    'Shopping_Mall': ['Forum Istanbul', 'Mall of Istanbul', 'Istinye Park']
})

pred_log = log_reg_pipeline.predict(ejemplos)
pred_rf = rf_pipeline.predict(ejemplos)

resultados = ejemplos.copy()
resultados['Pred_LogReg'] = pred_log
resultados['Pred_RF'] = pred_rf

resultados


,Edad,Genero,Categoria_Producto,Metodo_Pago,Shopping_Mall,Pred_LogReg,Pred_RF
0,22,Female,Clothing,Cash,Forum Istanbul,Alto,Alto
1,40,Male,Technology,Credit Card,Mall of Istanbul,Alto,Medio
2,65,Female,Shoes,Debit Card,Istinye Park,Alto,Alto


## 11) Importancia de variables

La importancia de variables permite entender qué características influyen más
en la clasificación del nivel de gasto del cliente, aportando interpretabilidad
al modelo.

In [27]:
feature_names = (
    num_features +
    list(rf_pipeline.named_steps['preprocessor']
         .named_transformers_['cat']
         .get_feature_names_out(cat_features))
)

importances = rf_pipeline.named_steps['classifier'].feature_importances_

pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)


,Feature,Importance
0,Edad,0.241441
6,Categoria_Producto_Food & Beverage,0.199036
4,Categoria_Producto_Clothing,0.173834
7,Categoria_Producto_Shoes,0.090122
5,Categoria_Producto_Cosmetics,0.069475
8,Categoria_Producto_Souvenir,0.063228
10,Categoria_Producto_Toys,0.051480
9,Categoria_Producto_Technology,0.049672
3,Categoria_Producto_Books,0.035924
11,Metodo_Pago_Cash,0.002240


### Interpretación de negocio

- La edad del cliente es un factor relevante para diferenciar niveles de gasto.
- Ciertas categorías de producto tienen mayor influencia en el gasto esperado.
- El método de pago y el centro comercial aportan información sobre el perfil
  económico del cliente.


## 12) Optimización de hiperparámetros - Random Forest

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2']
}

In [30]:
grid_search = GridSearchCV(
    rf_pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

/usr/local/python/3.12.1/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 